# Extract parameters of Interest



In [ ]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = 'movies.csv'
movies = pd.read_csv(file_path)

In [ ]:
# Extract movies-related columns
movies_data = movies[[
    'id', 'title', 'tagline', 'overview', 'original_language',
    'release_date', 'runtime', 'vote_average', 'vote_count', 'budget'
]]

# Rename columns to match database schema
movies_data.columns = [
    'movieId', 'title', 'tagline', 'overview', 'language',
    'releaseDate', 'runtime', 'voteAverage', 'voteCount', 'budget'
]

# Save to CSV for import
movies_data.to_csv('/mnt/data/movies_data.csv', index=False)
print("Movies data saved.")


# Process JSON Objects

## Movie Genre Processing

In [2]:
import json

In [ ]:
# Replace single quotes with double quotes for proper JSON parsing
movies['genres'] = movies['genres'].str.replace("'", '"', regex=False)

# Placeholder for genres list
genres_list = []

# Process each row in the movies DataFrame
for _, row in movies.iterrows():
    if pd.notna(row['genres']):
        try:
            # Parse genres JSON
            genres = json.loads(row['genres'])
            for genre in genres:
                genres_list.append({'movieId': row['id'], 'genreId': genre['id'], 'name': genre['name']})
        except json.JSONDecodeError:
            print(f"Skipping malformed JSON: {row['genres']}")

# Convert genres list to DataFrame
genres_df = pd.DataFrame(genres_list)

# Create unique Genres table
genres_table = genres_df[['genreId', 'name']].drop_duplicates()

# Explicitly convert genreId to integer for consistent sorting
genres_table['genreId'] = genres_table['genreId'].astype(int)

# Sort Genres table by genreId
genres_table = genres_table.sort_values(by='genreId')

# Save sorted Genres table
genres_table.to_csv('genres.csv', index=False)
print("Genres table saved and sorted by genreId.")

# Create MovieGenres table
movie_genres_table = genres_df[['movieId', 'genreId']].drop_duplicates()

# Remove rows with invalid movieId or genreId
movie_genres_table = movie_genres_table[
    movie_genres_table['movieId'].astype(str).str.isnumeric() &
    movie_genres_table['genreId'].astype(str).str.isnumeric()
]

# Convert columns to integers for consistent sorting
movie_genres_table['movieId'] = movie_genres_table['movieId'].astype(int)
movie_genres_table['genreId'] = movie_genres_table['genreId'].astype(int)

# Sort MovieGenres table by movieId and genreId
movie_genres_table = movie_genres_table.sort_values(by=['movieId', 'genreId'])

# Save sorted MovieGenres table
movie_genres_table.to_csv('movie_genres.csv', index=False)
print("MovieGenres table saved and sorted by movieId and genreId.")

Genres table saved and sorted by genreId.
MovieGenres table saved and sorted by movieId and genreId.


## Movie Producer Company Processing


In [ ]:
import ast

# Placeholder for company list
companies_list = []

# Ensure all values in production_companies are strings
movies['production_companies'] = movies['production_companies'].astype(str)

# Filter rows that look like valid JSON arrays
valid_rows = movies['production_companies'].str.startswith('[')
movies = movies[valid_rows]

# Process each row in the valid movies DataFrame
for _, row in movies.iterrows():
    try:
        # Safely evaluate the Python-like string
        companies = ast.literal_eval(row['production_companies'])

        # Process each company
        for company in companies:
            companies_list.append({'movieId': row['id'], 'companyId': company['id'], 'name': company['name']})
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Skipping malformed data: {row['production_companies']} - Error: {e}")

# Convert companies list to DataFrame
companies_df = pd.DataFrame(companies_list)

# Ensure movieId and companyId are numeric, drop invalid rows
companies_df['movieId'] = pd.to_numeric(companies_df['movieId'], errors='coerce')
companies_df['companyId'] = pd.to_numeric(companies_df['companyId'], errors='coerce')
companies_df = companies_df.dropna(subset=['movieId', 'companyId']).astype({'movieId': 'int', 'companyId': 'int'})

# Create unique Companies table
companies_table = companies_df[['companyId', 'name']].drop_duplicates().sort_values(by='companyId')
companies_table.to_csv('companies_sorted.csv', index=False)
print("Companies table saved and sorted by companyId.")

# Create MovieCompanies table
movie_companies_table = companies_df[['movieId', 'companyId']].drop_duplicates().sort_values(by=['movieId', 'companyId'])
movie_companies_table.to_csv('movie_companies_sorted.csv', index=False)
print("MovieCompanies table saved and sorted by movieId and companyId.")

Companies table saved and sorted by companyId.
MovieCompanies table saved and sorted by movieId and companyId.


## Production Country Processing

In [ ]:
# Placeholder for country list
countries_list = []

# Ensure all values in production_countries are strings
movies['production_countries'] = movies['production_countries'].astype(str)

# Filter rows that look like valid JSON arrays
valid_rows = movies['production_countries'].str.startswith('[')
movies = movies[valid_rows]

# Process each row in the valid movies DataFrame
for _, row in movies.iterrows():
    try:
        # Safely evaluate the Python-like string
        countries = ast.literal_eval(row['production_countries'])

        # Process each country
        for country in countries:
            countries_list.append({'movieId': row['id'], 'name': country['name']})
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Skipping malformed data: {row['production_countries']} - Error: {e}")

# Convert countries list to DataFrame
countries_df = pd.DataFrame(countries_list)

# Ensure movieId is numeric, drop invalid rows
countries_df['movieId'] = pd.to_numeric(countries_df['movieId'], errors='coerce')
countries_df = countries_df.dropna(subset=['movieId']).astype({'movieId': 'int'})

# Create unique Countries table with auto-generated countryId
countries_table = countries_df[['name']].drop_duplicates().reset_index(drop=True)
countries_table['countryId'] = countries_table.index + 1  # Generate sequential IDs
countries_table = countries_table[['countryId', 'name']]

# Save sorted Countries table
countries_table.to_csv('countries.csv', index=False)
print("Countries table saved with countryId and sorted by name.")

# Create MovieCountries table
movie_countries_table = countries_df.merge(countries_table, on='name')[['movieId', 'countryId']].drop_duplicates()
movie_countries_table = movie_countries_table.sort_values(by=['movieId', 'countryId'])

# Save sorted MovieCountries table
movie_countries_table.to_csv('movie_countries.csv', index=False)
print("MovieCountries table saved and sorted by movieId and countryId.")


Countries table saved with countryId and sorted by name.
MovieCountries table saved and sorted by movieId and countryId.


## Keywords processing

In [ ]:
# Load the keywords CSV
keywords_csv = pd.read_csv('keywords.csv')

# Placeholder for keywords list
keywords_list = []

# Ensure all values in keywords are strings
keywords_csv['keywords'] = keywords_csv['keywords'].astype(str)

# Filter rows that look like valid JSON arrays
valid_rows = keywords_csv['keywords'].str.startswith('[')
keywords_csv = keywords_csv[valid_rows]

# Process each row in the keywords CSV
for _, row in keywords_csv.iterrows():
    try:
        # Safely evaluate the Python-like string
        keywords = ast.literal_eval(row['keywords'])

        # Process each keyword
        for keyword in keywords:
            keywords_list.append({'movieId': row['id'], 'keywordId': keyword['id'], 'name': keyword['name']})
    except (ValueError, SyntaxError, TypeError) as e:
        print(f"Skipping malformed data: {row['keywords']} - Error: {e}")

# Convert keywords list to DataFrame
keywords_df = pd.DataFrame(keywords_list)

# Ensure movieId and keywordId are numeric, drop invalid rows
keywords_df['movieId'] = pd.to_numeric(keywords_df['movieId'], errors='coerce')
keywords_df['keywordId'] = pd.to_numeric(keywords_df['keywordId'], errors='coerce')
keywords_df = keywords_df.dropna(subset=['movieId', 'keywordId']).astype({'movieId': 'int', 'keywordId': 'int'})

# Create unique Keywords table
keywords_table = keywords_df[['keywordId', 'name']].drop_duplicates().sort_values(by='keywordId')
keywords_table.to_csv('keywords.csv', index=False)
print("Keywords table saved and sorted by keywordId.")

# Create MovieKeywords table
movie_keywords_table = keywords_df[['movieId', 'keywordId']].drop_duplicates().sort_values(by=['movieId', 'keywordId'])
movie_keywords_table.to_csv('movie_keywords.csv', index=False)
print("MovieKeywords table saved and sorted by movieId and keywordId.")

Keywords table saved and sorted by keywordId.
MovieKeywords table saved and sorted by movieId and keywordId.


## Rating Processing

In [4]:
# TODO
import pandas as pd

# Load the ratings dataset
ratings = pd.read_csv('ratings_small.csv')

# Extract relevant columns
ratings_filtered = ratings[['userId', 'movieId', 'rating']]

# Save the filtered data to a new CSV file
ratings_filtered.to_csv('ratings.csv', index=False)
print("Filtered ratings data saved successfully.")

Filtered ratings data saved successfully.
